In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


**Part One: Load the Datasets****

In [4]:
phone_url = "https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/cell_phones.csv"
population_url = "https://raw.githubusercontent.com/zalihat/cell_phone_dataset/master/population1.csv"
geo_url = "https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv"

In [5]:
phone_df = pd.read_csv(phone_url)
population_df = pd.read_csv(population_url)
geo_df = pd.read_csv(geo_url)

print('Successfully read in the 3 dataset!!')

Successfully read in the 3 dataset!!


**Part Two: Join Data**

In [7]:
population_df.head()

,Country,year,population
0,Afghanistan,1800,3280000
1,Afghanistan,1801,3280000
2,Afghanistan,1802,3280000
3,Afghanistan,1803,3280000
4,Afghanistan,1804,3280000


In [9]:
# Join the cell phone and the population df with inner join
population_phone_df = population_df.merge(phone_df, how='inner', on=['Country', 'year'])

In [10]:
population_phone_df.head()

,Country,year,population,cell_phones_total
0,Afghanistan,1960,9000000,0.0
1,Afghanistan,1961,9170000,0.0
2,Afghanistan,1962,9350000,0.0
3,Afghanistan,1963,9540000,0.0
4,Afghanistan,1964,9740000,0.0


In [11]:
population_phone_df.shape

(11640, 4)

In [14]:
# rename the country to geo in Geo_df and name to Country
geo_df = geo_df.rename(columns={'country': 'geo', 'name': 'Country'})

In [17]:
# select the geo and Country columns
geo_country_df = geo_df[['geo', 'Country']]

# merge with the population_phone_df
population_phone_geo_df = population_phone_df.merge(geo_country_df, how='inner', on='Country')

# shape
print(population_phone_geo_df.shape)

(11640, 5)


**Part Three: Make Features**

In [18]:
population_phone_geo_df.head()

,Country,year,population,cell_phones_total,geo
0,Afghanistan,1960,9000000,0.0,afg
1,Afghanistan,1961,9170000,0.0,afg
2,Afghanistan,1962,9350000,0.0,afg
3,Afghanistan,1963,9540000,0.0,afg
4,Afghanistan,1964,9740000,0.0,afg


In [19]:
# Calculate the number of cell phones per person, and add this column onto df
population_phone_geo_df['cell_per_person'] = population_phone_geo_df['cell_phones_total'] / population_phone_geo_df['population']


In [32]:
a = population_phone_geo_df.groupby(['year','Country'])['cell_per_person'].count()

In [33]:
a = pd.DataFrame(a.reset_index())


In [39]:
# modify the geo column to UPPER case
population_phone_geo_df['geo'] = [str.upper(x) for x in population_phone_geo_df['geo']]

In [40]:
population_phone_geo_df.head()

,Country,year,population,cell_phones_total,geo,cell_per_person
0,Afghanistan,1960,9000000,0.0,AFG,0.0
1,Afghanistan,1961,9170000,0.0,AFG,0.0
2,Afghanistan,1962,9350000,0.0,AFG,0.0
3,Afghanistan,1963,9540000,0.0,AFG,0.0
4,Afghanistan,1964,9740000,0.0,AFG,0.0


**Part Four: Process Data**

In [41]:
# describe numeric column
population_phone_geo_df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,11640.0,1.989500e+03,1.731885e+01,1960.0,1974.75,1989.5,2.004250e+03,2.019000e+03
population,11640.0,2.710307e+07,1.086187e+08,4380.0,1117500.00,5100000.0,1.590000e+07,1.430000e+09
cell_phones_total,11640.0,7.875722e+06,5.572496e+07,0.0,0.00,0.0,5.615000e+05,1.730000e+09
cell_per_person,11640.0,2.355361e-01,4.333029e-01,0.0,0.00,0.0,2.336673e-01,2.126068e+00


In [42]:
# descrube non-numeric
population_phone_geo_df.describe(include='object')

,Country,geo
count,11640,11640
unique,194,194
top,Eritrea,JAM
freq,60,60


In [47]:
assert 194 == population_phone_geo_df.describe(include='object')['Country']['unique']

In [53]:
# In 2017, what were the top 5 countries with the most cell phones total?

df_2017 = population_phone_geo_df[population_phone_geo_df['year'] == 2017]

top_5_countries_phone_tot = df_2017.groupby(['Country'])['cell_phones_total'].sum().sort_values(ascending=False)

In [57]:
top_5_countries_phone_tot = pd.DataFrame(top_5_countries_phone_tot.reset_index())

top_5_countries_phone_tot.style.background_gradient()

,Country,cell_phones_total
0,China,1470000000.000000
1,India,1170000000.000000
2,Indonesia,435000000.000000
3,United States,400000000.000000
4,Russia,227000000.000000
5,Brazil,218000000.000000
6,Japan,173000000.000000
7,Bangladesh,151000000.000000
8,Nigeria,145000000.000000
9,Pakistan,145000000.000000


In [59]:
top_5_countries_phone_tot.head().style.background_gradient()

,Country,cell_phones_total
0,China,1.470000e+09
1,India,1.170000e+09
2,Indonesia,4.350000e+08
3,United States,4.000000e+08
4,Russia,2.270000e+08
